<hr style="height:2px;border:none" />

# pydvma logger template

*A template for using pydvma, the python dynamics and vibration measurement and analysis package*

<hr style="height:2px;border:none" />

Start by importing the necessary modules (pydvma is a python package written for data acquisition at CUED).

In [1]:
# If the import doesn't work below, then uncomment and run the line below first.
# The logger uses pyqt5: sometimes you need to set this backend explicity.
# %matplotlib qt

In [1]:
import pydvma as ma
import matplotlib
import numpy as np

In [2]:
%matplotlib widget

In [3]:
d = ma.create_test_impulse_data_nonlinear_v2()
d.save_data()

2025-09-15 12:44:52.047 python[26506:1388942] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


Data saved as /Users/tore/Library/CloudStorage/OneDrive-UniversityofCambridge/Work Teaching - onedrive/4C6/2025/Lectures Notes/media/nonlinear_2.npy


'/Users/tore/Library/CloudStorage/OneDrive-UniversityofCambridge/Work Teaching - onedrive/4C6/2025/Lectures Notes/media/nonlinear_2.npy'

<hr style="height:2px;border:none" />

## Oscilloscope

<hr style="height:2px;border:none" />

Choose your acquisition settings:

* channels=2 (number of channels to record)
* fs=44100 (sampling rate in Hz)
* chunk_size=200 (how many samples to collect at a time, effectively controls refresh rate of oscilloscope)
* stored_time=2 (time in seconds to record data for)
* viewed_time=2 (time in seconds to display on oscilloscope)
* device_index = 1 (Windows default input)

In [ ]:
settings = ma.MySettings(channels=1,
                           fs=44100,
                           chunk_size=200,
                           stored_time=2,
                           viewed_time=2,
                           device_driver='soundcard')
# when using national instruments acquisition cards, change device_driver='nidaq'

Now open a PC oscilloscope using your settings. This shows three plots:

* the top one is like a normal oscilloscope showing the signal (toggle on/off with 'T');
* the middle one shows the frequency spectrum of the signal (toggle on/off with 'F');
* the bottom one shows the signal amplitudes (toggle on/off with 'L');
* you can pause the data shown by pressing 'P' (press again to continue streaming data);
* you can toggle whether the window is always on top by pressing 'A'

Press the **space bar** to record data from the past 'stored_time' seconds.

* The first time you press it you will be prompted for where to save your data.
* Use the save dialog to navigate to where you want to save your data
* Subsequent times you press it will auto-save to the same folder with a number added to the filename.
* Press 's' if you want to save data to a new filename or location. Pressing space after that will auto-save with the new name.

**Note that pressing 'space' captures the past N seconds of data: so you need to tap the beam, wait a second or so, then press space!**

In [ ]:
osc = ma.Oscilloscope(settings)

<hr style="height:2px;border:none" />

## Logger GUI

<hr style="height:2px;border:none" />

You can also log data with the logger gui:

In [ ]:
# COMMON SETTINGS EXAMPLE 1 (normal acquisition)
settings = ma.MySettings(channels=1,
                           fs=44100,
                           stored_time=2,
                           device_driver = 'soundcard')
logger1 = ma.Logger(settings,
                      test_name = 'ENTER_YOUR_TEST_NAME',
                      default_window='hann')


||PaMacCore (AUHAL)|| AUHAL component not found.__________________________________________________________

Devices available using device_driver=soundcard, by index:
__________________________________________________________

0: TB phone Microphone
1: BlackHole 2ch
2: MacBook Pro Microphone
3: MacBook Pro Speakers
4: Tore
5: Microsoft Teams Audio

Default device is: [1] BlackHole 2ch

Default device is: [3] MacBook Pro Speakers


______________________________________________________

Devices available using device_driver=nidaq, by index:
______________________________________________________

no NI devices found

||PaMacCore (AUHAL)|| AUHAL component not found.__________________________________________________________

Devices available using device_driver=soundcard, by index:
__________________________________________________________

0: TB phone Microphone
1: BlackHole 2ch
2: MacBook Pro Microphone
3: MacBook Pro Speakers
4: Tore
5: Microsoft Teams Audio

Default device is: [1] Bla

PortAudioError: Error opening InputStream: Unanticipated host error [PaErrorCode -9999]: '' [<host API not found> error 0]

: 

In [ ]:
# COMMON SETTINGS EXAMPLE 2 (acquisition with pre-trigger)
settings = ma.MySettings(channels=2,
                           fs=3000,
                           stored_time=2,
                           pretrig_samples=100,
                           device_driver = 'soundcard',
                           device_index=1)
logger2 = ma.Logger(settings,
                      test_name = 'ENTER_YOUR_TEST_NAME',
                      default_window=None)

<div class="alert alert-block alert-info">
    
<b> Understanding the logger: </b><br/>

When you press 'Save Dataset', the logger saves a &lt;DataSet&gt; object which has the following structure:

<pre><code>
    &lt;DataSet> class:
          time_data_list: [&lt;TimeData>, &lt;TimeData>, &lt;TimeData>]
          freq_data_list: [&lt;FreqData>, &lt;FreqData>, &lt;FreqData>]
    cross_spec_data_list: []
            tf_data_list: [&lt;TfData>, &lt;TfData>, &lt;TfData>]
         modal_data_list: [&lt;ModalData>]
          sono_data_list: []
          meta_data_list: []
          
</code></pre>

    
Each type of data is arranged in 'sets', e.g. each measurement will add another &lt;TimeData&gt; set to time_data_list. This example has three sets of time, frequency and transfer function data types, and a &lt;ModalData&gt; set. Similarly each time you load data it will add the loaded sets to the appropriate data list.<br/><br/>

<li> When you press 'Calc FFT' then a &lt;FreqData&gt; item is calculated for each &lt;TimeData&gt; set.</li>
<li> When you press 'Calc TF' then a &lt;TfData&gt; item is calculated for each &lt;TimeData&gt; set.</li>
<li> When you press 'Calc TF average' then a single &lt;TfData&gt; item is calculated averaging across all &lt;TimeData&gt; items, with an assumption that the time data sets all have the same settings.</li>
</div>

<hr style="height:2px;border:none" />

## Command line tools

<hr style="height:2px;border:none" />

Another way to log data directly from the command line:

In [ ]:
# SETTINGS
settings = ma.MySettings(channels=1,
                           fs=44100,
                           stored_time=2,
                           device_driver = 'soundcard')
d = ma.log_data(settings)

In [ ]:
d.plot_time_data()

Calculate the FFT or transfer function:

In [ ]:
d.calculate_fft_set(window='hanning')
d.calculate_tf_set(window='hanning',N_frames=3)
d.plot_freq_data()
d.plot_tf_data()

Save / load the dataset:

In [ ]:
ma.save_data(d)

In [ ]:
d = ma.load_data()